In [20]:
import numpy as np
import pandas as pd
import numpy_financial as npf
from datetime import datetime
import matplotlib.pyplot as plt

### Load stock data

In [21]:
# Get list of stock names
names = pd.read_csv("../data/stock_list_full.csv", header=None, names=['code'])
names = list(names['code'].str.lower())
ten_names = ['aaa','vnm','fpt','pnj','lix','ssi','vic','vcb','vnindex','vn30index']

In [22]:
# Get data of each stock
def get_data(code, path):
    f = path+"/"+code+".jl"    
    df = pd.read_json(f, lines=True)
    df['transc_date'] =pd.to_datetime(df['transc_date'], dayfirst=True)
    df.index = df['transc_date']
    return df

def filter_data(df, start=None, end=None):
    df = df[(df['transc_date'] >= start) & (df['transc_date'] <= end)]
    return df

In [23]:
path = "../data"
sta = datetime(2015,4,20)
end = datetime(2020,4,20)

In [24]:
df1 = get_data('hpg', path)
df1.shape

(3324, 8)

In [25]:
df = filter_data(df1, sta, end)

In [26]:
df.shape

(1249, 8)

### Calculate RoIs

In [27]:
def daily_rate(df):
    """
    Get daily rate from a serie of adjusted/closed prices
    """
    n = df.shape[0] - 1    
    sta_price = float(str(df[0]).replace(',',''))
    end_price = float(str(df[n]).replace(',',''))
    return (end_price/sta_price)**(1/n) - 1

def yearly_rate(daily_rate):
    """Get yearly rate from a daily rate"""
    return (1+daily_rate)**(249.8) - 1

def get_rois(stk_names, start, end):
    d_rois = []
    for code in stk_names:
        df = get_data(code,path)
        data = filter_data(df, start, end)['adjted_price']
        if data.shape[0] > 0:
            roi = daily_rate(data)
            d_rois.append([code, roi])
    df = pd.DataFrame(d_rois, columns=['code','daily_roi'])
    df['yearly_roi'] = df['daily_roi'].apply(yearly_rate)
    return df

In [28]:
# Set window period
sta = datetime(2017,1,1)
end = datetime(2020,1,1)
df_rois = get_rois(names, sta, end)

In [30]:
# Top stock performance
df_rois.sort_values('yearly_roi', ascending=False).head(15)

,code,daily_roi,yearly_roi
10,ctd,0.001582,0.484069
64,tcb,0.000933,0.262237
4,bmp,0.000872,0.243321
53,pow,0.000869,0.242435
66,tpb,0.000823,0.228092
26,hdb,0.000661,0.179563
41,ntp,0.000638,0.172652
69,vci,0.000508,0.135225
51,pme,0.000408,0.107266
0,aaa,0.000365,0.095484
